In [1]:
# Install Python for Kafka if necessary: pip install kafka-python
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import requests
import json

# Create the producer object on the EC2 public IP where Kafka is running
producer = KafkaProducer(bootstrap_servers=['3.20.222.206:9092'], 
                         value_serializer=lambda x: dumps(x).encode('utf-8')) # UTF-8 encoding to JSON format

In [9]:
# Test running of Kafka server
producer.send('test1', value="{'name': 'Ziyang', 'message': 'Hello World'}")

In [ ]:
API_KEY = 'MY_IEX_CLOUD_API'
BASE_URL = 'https://sandbox.iexapis.com/stable/'
RATE_LIMIT = 5

def get_stock_data(symbol):
    try:
        url = f"{BASE_URL}stock/{symbol}/quote?token={API_KEY}"
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        return response.json() # Convert to proper JSON structure
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error for {symbol}: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Error Connecting for {symbol}: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error for {symbol}: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Error for {symbol}: {err}")
        
    return None

stocks = ["AAPL", "GOOGL", "TSLA", "MSFT", "AMZN"]
while True:
    for stock in stocks:
        stock_data = get_stock_data(stock)
        stock_data['ticker'] = stock
        if stock_data:
            producer.send('test1', value=stock_data)

    sleep(60/RATE_LIMIT) # Set a break between each send to prevent overloading

In [ ]:
producer.flush() # Flush (empty) all data sent in Kafka if necessary